In [1]:
# basic packages
import os
import subprocess
import logging
from tqdm import tqdm
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from time import localtime, strftime

In [2]:
# sagemaker parameters
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
import boto3

session = sagemaker.Session()
role = get_execution_role()
bucket = session.default_bucket()
prefix = 'kaggle/tweetSentiment'  # Prefix should not tontain '/' at the end!
s3 = boto3.client('s3')

In [3]:
# directories
working_dir = '/home/ec2-user/SageMaker/kaggle_data'
data_dir = os.path.join(working_dir, 'processed_data/')
output_dir = os.path.join(working_dir, 'output/')
if not os.path.exists(output_dir):
    subprocess.check_call('mkdir {}'.format(output_dir), shell=True)

In [4]:
with open('tf_model.txt', 'r') as f:
    tf_model = f.read().split()[0]
print(tf_model)


tensorflow-training-200610-1950-009-1915ccbb


In [5]:
from sagemaker.tensorflow.model import TensorFlowModel

In [6]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(
    entry_point='train.py',
    source_dir='source',
    role=role,
    train_instance_count=1,
    train_instance_type='ml.p2.xlarge',
    framework_version='2.1.0',
    py_version='py3',
    distributions={'parameter_server': {'enabled': True}},
)

Parameter distribution will be renamed to {'parameter_server': {'enabled': True}} in SageMaker Python SDK v2.


In [7]:
tf_estimator = tf_estimator.attach(tf_model)

2020-06-10 20:55:00 Starting - Preparing the instances for training
2020-06-10 20:55:00 Downloading - Downloading input data
2020-06-10 20:55:00 Training - Training image download completed. Training in progress.
2020-06-10 20:55:00 Uploading - Uploading generated training model
2020-06-10 20:55:00 Completed - Training job completed2020-06-10 20:53:01,664 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-06-10 20:53:01,665 sagemaker-containers INFO     Failed to parse hyperparameter _tuning_objective_metric value loss to Json.
Returning the value itself
2020-06-10 20:53:01,695 sagemaker_tensorflow_container.training INFO     Appending the training job name to model_dir: s3://sagemaker-us-east-2-815596061983/kaggle/tweetSentiment/model/tensorflow-training-2020-06-10-19-50-44-027/model/tensorflow-training-200610-1950-009-1915ccbb/model
2020-06-10 20:53:02,140 sagemaker-containers INFO     Installing module with the following command:
/usr/bin/p

Epoch 11/100
#015 128/5709 [..............................] - ETA: 0s - loss: 0.0269 - accuracy: 0.9844#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 768/5709 [===>..........................] - ETA: 0s - loss: 0.0510 - accuracy: 0.9661#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0151664/5709 [=======>......................] - ETA: 0s - loss: 0.0568 - accuracy: 0.9627#010#010#010#010#010#010#010#

In [8]:
predictor = tf_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-------------!

In [9]:
test = pd.read_csv('processed_data/test_processed.csv', header=None)
test = test.to_numpy()


In [10]:
# Sending all the test data at once returns 'Broken Pipe' error.
# So send them by chunks.

def predict_by_chunks(test_arr, chunk_size):
    test_split = np.array_split(test_arr, test_arr.shape[0] / chunk_size)
    for i in range(len(test_split)):
        result = predictor.predict(test_split[i])
        result = np.array(result['predictions'])
        if i == 0:
            result_all = result.copy()

        else:
            result_all = np.append(result_all, result)
            
    return result_all.astype(int)

In [11]:
y_pred = predict_by_chunks(test, 100)

In [12]:
test = pd.read_csv("test.csv")

In [13]:
out = pd.concat([test.id, pd.DataFrame(y_pred)], axis=1)
out.columns = ['id', 'target']

In [14]:
out.to_csv("output/tf_out.csv", header=True, index=False)
subprocess.check_call('aws s3 cp {} s3://{}/{}/tf_out.csv'.format(os.path.join(output_dir, 'tf_out.csv'), bucket, prefix), shell=True)

0

In [15]:
# Print a path to s3 location.
os.path.join('s3://', bucket, prefix, 'tf_out.csv')

's3://sagemaker-us-east-2-815596061983/kaggle/tweetSentiment/tf_out.csv'

In [16]:
predictor.delete_endpoint()